# Task Overview 
Task: In this synthetic dataset based off of a real dataset funded by the Mayo Clinic, each example represents both general and survival information about a patient that has liver cirrhosis, a condition involving prolonged liver damage. The goal is to train a machine learning model that can predict the patient's current survival status based on the data features. 

Approach: Our approach will be to train a Logistic Regression model using sklearn to be used as a baseilne model and to then train a performance-focused model using XGBoost as a learning exercise.  

Conclusions: 

In [1]:
import numpy as np
import pandas as pd

# Read in the training data as a pandas df 
train_file_path = "/kaggle/input/playground-series-s3e26/train.csv"
train_df = pd.read_csv(train_file_path) 

# test_file_path = 

train_df.head(5)

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [2]:
train_df.info()

# Make a list of categorical columns for future use (in the feature scaling section)
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Status']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7905 non-null   int64  
 1   N_Days         7905 non-null   int64  
 2   Drug           7905 non-null   object 
 3   Age            7905 non-null   int64  
 4   Sex            7905 non-null   object 
 5   Ascites        7905 non-null   object 
 6   Hepatomegaly   7905 non-null   object 
 7   Spiders        7905 non-null   object 
 8   Edema          7905 non-null   object 
 9   Bilirubin      7905 non-null   float64
 10  Cholesterol    7905 non-null   float64
 11  Albumin        7905 non-null   float64
 12  Copper         7905 non-null   float64
 13  Alk_Phos       7905 non-null   float64
 14  SGOT           7905 non-null   float64
 15  Tryglicerides  7905 non-null   float64
 16  Platelets      7905 non-null   float64
 17  Prothrombin    7905 non-null   float64
 18  Stage   

In [3]:
train_df.describe()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000
mean,3952.000000,2030.173308,18373.146490,2.594485,350.561923,3.548323,83.902846,1816.745250,114.604602,115.340164,265.228969,10.629462,3.032511
std,2282.121272,1094.233744,3679.958739,3.812960,195.379344,0.346171,75.899266,1903.750657,48.790945,52.530402,87.465579,0.781735,0.866511
min,0.000000,41.000000,9598.000000,0.300000,120.000000,1.960000,4.000000,289.000000,26.350000,33.000000,62.000000,9.000000,1.000000
25%,1976.000000,1230.000000,15574.000000,0.700000,248.000000,3.350000,39.000000,834.000000,75.950000,84.000000,211.000000,10.000000,2.000000
50%,3952.000000,1831.000000,18713.000000,1.100000,298.000000,3.580000,63.000000,1181.000000,108.500000,104.000000,265.000000,10.600000,3.000000
75%,5928.000000,2689.000000,20684.000000,3.000000,390.000000,3.770000,102.000000,1857.000000,137.950000,139.000000,316.000000,11.000000,4.000000
max,7904.000000,4795.000000,28650.000000,28.000000,1775.000000,4.640000,588.000000,13862.400000,457.250000,598.000000,563.000000,18.000000,4.000000


## Dataset Pre-Processing
Task Overview
1. Target Variable Label Encoding 
2. Missing Feature Value Handling (Unnecessary for XGBoost)
3. Feature Scaling (Unnecessary for XGBoost)
4. Categorical Attribute Handling (Should be done at the end since it may change the number of columns)
5. Train/Test Split

# Target Variable Label Encoding

Problem: Since the target column (Status) is a text column, we need to convert the values into integers.

Approaches:
1. Label Encoding - Convert each text category into an integer label. 
2. Ordinal Encoding - Convert each text category into an integer label but with a particular order. Used when the categories have some quantitative order that can be taken advantage of.   
3. One-Hot Encoding - Convert each text category into a separate column. For example, this is done in the softmax layer of a neural network. 

Solution: There is no obvious ordering in the 'Status' column and ML libraries typically expect a single target column so we will use label encoding.

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder() 
target_col_encoded = label_encoder.fit_transform(train_df['Status'])

train_df['Status'] = target_col_encoded 

## Missing Feature Value Handling

Problem: We need to check if there are any missing feature values and handle the problem accordingly.

Solution: Since we are using XGBoost, we do not need to do dataset pre-processing for missing feature values since XGBoost's implementation automatically handles them. We still check for missing feature values for practice by using the isna DataFrame method which returns a boolean DataFrame where each cell is True if the value is missing and False otherwise. We then apply the sum method to find the number of missing values in each column. 

In [5]:
# Check for missing feature values
print("Number of missing feature values by column: ")
print(train_df.isna().sum())

Number of missing feature values by column: 
id               0
N_Days           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
Status           0
dtype: int64


## Feature Scaling

Problem: Typically since feature column values are combined to create the final classification, the ML model will perform better if the features are on the same scale. 

Approaches: 
1. Min-Max Scaling - Scales the data to a fixed range between two values (typically 0 and 1). Most useful for neural networks.
2. Standardization (Z-score Normalization) - Scales the data so that the mean is 0 and the standard deviation is 1. Most useful for algorithms that assume a normal distribution of data, such as SVMs and logistic regression.
3. Robust Scaling - Scaling based on median and IQR. Most useful for handling significant outliers
4. MaxAbsScaler - Scales each feature based on its maximum absolute value. Useful for sparse data. 

Solution: Since XGBoost's decision tree classification uses splitting which occurs within a column, different column values do not interact with each other and therefore scaling the features is not necessary. However, since we are using Logistic Regression as our baseline model and features do interact in training we will need to do feature scaling. Since we doing feature scaling specifically for our logistic regression model we will use the Standardization approach. 

In [6]:
from sklearn.preprocessing import StandardScaler 

numerical_cols = train_df.columns.difference(categorical_cols)

std_scaler = StandardScaler() 
train_df[numerical_cols] = std_scaler.fit_transform(train_df[numerical_cols])
                      
# Confirm the transformation was successful by seeing if the mean = 0 and std = 1 for numerical columns
train_df.describe()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
count,7905.000000,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7.905000e+03,7905.000000
mean,0.000000,-5.393114e-17,-4.602124e-16,-5.482999e-17,6.291966e-18,-5.285251e-16,-2.966213e-17,-5.393114e-18,3.775180e-16,-6.606564e-17,-2.139268e-16,5.959391e-16,-2.372970e-16,0.709045
std,1.000063,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,0.938439
min,-1.731832,-1.817984e+00,-2.384728e+00,-6.017974e-01,-1.180148e+00,-4.588553e+00,-1.052815e+00,-8.025431e-01,-1.808946e+00,-1.567576e+00,-2.323678e+00,-2.084550e+00,-2.345776e+00,0.000000
25%,-0.865916,-7.313099e-01,-7.606943e-01,-4.968854e-01,-5.249706e-01,-5.729401e-01,-5.916485e-01,-5.162480e-01,-7.922996e-01,-5.966478e-01,-6.200428e-01,-8.052631e-01,-1.191649e+00,0.000000
50%,0.000000,-1.820323e-01,9.235837e-02,-3.919734e-01,-2.690420e-01,9.151385e-02,-2.754199e-01,-3.339647e-01,-1.251254e-01,-2.158918e-01,-2.617984e-03,-3.769082e-02,-3.752190e-02,0.000000
75%,0.865916,6.021276e-01,6.279961e-01,1.063586e-01,2.018666e-01,6.404106e-01,2.384516e-01,2.114631e-02,4.785084e-01,4.504312e-01,5.805054e-01,4.740240e-01,1.116605e+00,2.000000
max,1.731832,2.526884e+00,2.792831e+00,6.663359e+00,7.291089e+00,3.153780e+00,6.642081e+00,6.327728e+00,7.023169e+00,9.188781e+00,3.404652e+00,9.429034e+00,1.116605e+00,2.000000


## Categorical Attributes/Columns 
Problem: We use the DataFrame info method to find that there are six categorical columns: drug, sex, ascites, hepatomegaly, spiders, edema which we need to convert from text into numerical values.

Solution: The main approaches for categorical attribute handling are 
1. Ordinal Encoding - Useful when the categories correspond to an ascending or descending order. 
2. One-Hot Encoding - For each categorical column, convert it into multiple columns, one for each possible category. This is used when the categories do not have an obvious logical order. 
3. Numerical Feature Replacement (Advanced) - In cases where the number of categories is cery large (hundreds or thousands) one should consider replacing the categorical columns with a numerical column that converts each category into some number. For example, one could convert a country code into the country's population. 
4. Embedding Replacement (Advanced) - Alternatively, one can replace categories with embeddings, which are low dimensional vectors that represent the category. 

In this case, we use a one-hot encoding since none of the categories seem to have a logical order and the number of categories is low (under 10 for all categorical columns)

In [7]:
# Confirm that the number of categories in the categorical columns is manageable (< 100)
unique_values_per_column = train_df.nunique()

print(unique_values_per_column)

id               7905
N_Days            461
Drug                2
Age               391
Sex                 2
Ascites             2
Hepatomegaly        2
Spiders             2
Edema               3
Bilirubin         111
Cholesterol       226
Albumin           160
Copper            171
Alk_Phos          364
SGOT              206
Tryglicerides     154
Platelets         227
Prothrombin        49
Stage               4
Status              3
dtype: int64


In [8]:
# Convert the categorical columns into one-hot encodings
status = train_df['Status']
train_df_dummies = pd.get_dummies(train_df.drop('Status', axis=1))
train_df = pd.concat([train_df_dummies, status], axis=1)

# Confirm the transformation was successful
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    7905 non-null   float64
 1   N_Days                7905 non-null   float64
 2   Age                   7905 non-null   float64
 3   Bilirubin             7905 non-null   float64
 4   Cholesterol           7905 non-null   float64
 5   Albumin               7905 non-null   float64
 6   Copper                7905 non-null   float64
 7   Alk_Phos              7905 non-null   float64
 8   SGOT                  7905 non-null   float64
 9   Tryglicerides         7905 non-null   float64
 10  Platelets             7905 non-null   float64
 11  Prothrombin           7905 non-null   float64
 12  Stage                 7905 non-null   float64
 13  Drug_D-penicillamine  7905 non-null   bool   
 14  Drug_Placebo          7905 non-null   bool   
 15  Sex_F                

# Train/Test Split

Problem: We need to split the dataset into a training dataset and a testing/tuning dataset.

Solution: We wait until all dataset pre-processing is done (to avoid needing to pre-process the train/test datasets separately) and then split the dataset using sklearn's train_test_split function. 

In [9]:
from sklearn.model_selection import train_test_split

# Split the training set into a training and test set 
X = train_df.drop("Status", axis=1)
y = train_df["Status"]

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size = 0.2)

# Logistic Regression Baseline Model Training Tasks
1. Train the Logistic Regression modle using sklearn
2. Evaluate the results 

In [10]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train the model 
model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the results
#print("Accuracy:", accuracy_score(y_test, y_pred))
#print("Classification Report:\n", classification_report(y_test, y_pred))
#print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [11]:
# Convert the predictions into a csv for submission
# final_sub_df.to_csv('submission.csv', index = False)